In [1]:
import torch
import clip
import os
import random
import numpy as np
from tqdm import tqdm
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from nltk.corpus import wordnet as wn
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR100, ImageNet
from google_images_search import GoogleImagesSearch
import pandas as pd
import PIL
import cv2
import pickle
import torchvision
from sklearn.metrics import mutual_info_score

In [2]:
import sqlite3


class BGVarDB:

    ANNOTATIONS_TABLE = "annotations"

    def __init__(self, path) -> None:
        self._connection = sqlite3.connect(path)
        self._cursor = self._connection.cursor()

        self._cursor.execute(
            f"CREATE TABLE if not exists {BGVarDB.ANNOTATIONS_TABLE} (file_name TEXT PRIMARY KEY, category TEXT, time TEXT, weather TEXT, locations TEXT, humans TEXT)",
        )

    def write_entry(self, file_name, category, time, weather, locations, humans):
        self._cursor.execute(
            f"REPLACE INTO {BGVarDB.ANNOTATIONS_TABLE} VALUES (?, ?, ?, ?, ?, ?)",
            (file_name, category, time, weather, ", ".join((sorted(locations))), humans),
        )
        self._connection.commit()

    def read_entries(self, categories=None, times=None, weathers=None, locations=None, humans=None):

        query = f"SELECT file_name, category, time, weather, locations FROM {BGVarDB.ANNOTATIONS_TABLE} WHERE "
        conditions = []
        if categories is not None:
            conditions.append(f"category IN {BGVarDB.stringify(categories)}")
        if times is not None:
            conditions.append(f"time IN {BGVarDB.stringify(times)}")
        if weathers is not None:
            conditions.append(f"weather IN {BGVarDB.stringify(weathers)}")
        if locations is not None:
            conditions.append(f"locations LIKE '%{', '.join((sorted(locations)))}%'")
        if humans is None:
            conditions.append(f"humans = 'no'")

        query += " AND ".join(conditions)
        results = self._cursor.execute(query)
        yield from results

    def __del__(self) -> None:
        self._connection.commit()
        self._connection.close()

    def populate_temp_table(self):
        # TODO: delete this later
        self.write_entry("cat/1.jpg", "cat", "day", "sunny", ["forest", "grass"])
        self.write_entry("cat/89.jpg", "cat", "day", "foggy", ["forest"])
        self.write_entry("dog/91.jpg", "dog", "day", "sunny", ["indoors"])
        self.write_entry("horse/19.jpg", "horse", "night", "none", ["grass", "water"])
        self.write_entry("car/23.jpg", "car", "day", "cloudy", ["snow", "street"])
        self.write_entry("plane/76.jpg", "plane", "day", "raining", ["street"])

    def check_if_table_exists(self, table_name):
        self._cursor.execute(
            f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'"
        )

        return self._cursor.fetchone()[0] != 0

    def clear_annotations(self):
        self._cursor.execute(f"DROP TABLE {BGVarDB.ANNOTATIONS_TABLE}")

    @staticmethod
    def stringify(values):
        if len(values) == 1:
            return f"('{values[0]}')"
        else:
            return str(tuple(values))


        
class BGVarDataset(torch.utils.data.Dataset):
    
    categories = {
        "truck": 0,
        "car": 1,
        "plane": 2,
        "ship": 3,
        "cat": 4,
        "dog": 5,
        "horse": 6,
        "deer": 7,
        "frog": 8,
        "bird": 9,
    }

    times = {
        "day": 0,
        "night": 1,
        "none": 2,
    }

    weathers = {
        "cloudy": 0,
        "foggy": 1,
        "partly cloudy": 2,
        "raining": 3,
        "snowing": 4,
        "sunny": 5,
        "none": 6,
    }

    locations = {
        "forest": 0,
        "grass": 1,
        "indoors": 2,
        "rocks": 3,
        "sand": 4,
        "street": 5,
        "snow": 6,
        "water": 7,
        "none": 8,
    }

    def __init__(
        self,
        root,
        database=None,
        categories=None,
        times=None,
        weathers=None,
        locations=None,
        humans="yes",  # TODO: change this later
        transform=None,
        target_transform=None,
    ) -> None:
        self.root = Path(root)
        self.transform = transform
        self.target_transform = target_transform
        if database is None:
            database = BGVarDB(self.root / "annotations.db")

        self.image_files = list(
            database.read_entries(
                categories=categories,
                times=times,
                weathers=weathers,
                locations=locations,
                humans=humans,
            )
        )

    def __getitem__(self, idx):
        image_path = self.root / (self.image_files[idx][0][1:])
        image = Image.open(image_path)
        if image.mode != "RGB":
            image = image.convert("RGB")

        category = BGVarDataset.categories[self.image_files[idx][1]]
        try:
            if self.transform:
                image = self.transform(image)
        except:
            return self.__getitem__(idx-1)
        if self.target_transform:
            category = self.target_transform(category)
        time = BGVarDataset.times[self.image_files[idx][2]]
        weather = BGVarDataset.weathers[self.image_files[idx][3]]
        locations = torch.zeros(len(BGVarDataset.locations), dtype=torch.long)
        for location in (self.image_files[idx][4]).split(", "):
            locations[BGVarDataset.locations[location]] = 1
        
        return image, category, time, weather, locations

    def __len__(self) -> int:
        return len(self.image_files)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clip_model, preprocess = clip.load('ViT-B/32', device)

In [25]:
"{} rferf {}".format("1", "2")

'1 rferf 2'

In [4]:
class VisualModel(torch.nn.Module):
    def __init__(self, normal_text_probes, clip_model, device, *all_context_text_probes):
        super(VisualModel, self).__init__()
        self.device = device
        self.model = clip_model
        self.all_text_context_format = "a photo of a  in the {} at {}"
        def text_encode(text_probes):
            texts = clip.tokenize(text_probes).to(device) #tokenize
            class_embeddings = self.model.encode_text(texts) #embed with text encoder
            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            return class_embeddings
        
        self.text_probes = text_encode(normal_text_probes)
        self.text_probes_with_context = [torch.stack([text_encode(tp) for tp in context_tp])\
                                         for context_tp in all_context_text_probes]
        
    def forward(self, images, which=None, context=None):
        image_features = self.model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        if which is None:
            text_features = self.text_probes
            logits = image_features @ text_features.transpose(0,1)
        elif which == 4:
            "a photo of a "
        else:    
            all_text_features = self.text_probes_with_context[which][context]
            logits = torch.matmul(image_features.unsqueeze(1), all_text_features.transpose(1,2))
        print(logits.shape)
#         print(image_features.shape, text_features.shape)
        return logits.squeeze(1)

In [5]:
dataset = BGVarDataset(
        Path("/cmlscratch/pkattaki/datasets/bg-var"),
        categories=BGVarDataset.categories,
        times=None,
        weathers=None,
        locations=None,
        transform=preprocess,
    )

In [6]:
dataloader = DataLoader(dataset, batch_size=256, shuffle=False, num_workers=0)

In [7]:
classes = list(BGVarDataset.categories.keys())
text_probes = ["a photo of a "+c for c in classes]
loc_text_probes = [["a photo of a "+c+" in the "+loc for c in classes] for loc in dataset.locations.keys() if loc is not None]
weather_text_probes = [["a photo of a "+c+" when it is "+w for c in classes] for w in dataset.weathers.keys() if w is not None]
time_text_probes = [["a photo of a "+c+" in the "+t for c in classes] for t in dataset.times.keys() if t is not None]
model = VisualModel(text_probes, clip_model, device, loc_text_probes, weather_text_probes, time_text_probes).to(device)

In [8]:
torch.cuda.empty_cache()

In [9]:
!nvidia-smi

Mon Nov 22 18:04:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:1B:00.0 Off |                  N/A |
| 32%   36C    P2    65W / 250W |   6622MiB / 11019MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
uncommon = {
    0: {  # truck
        "time": {1},
        "weather": {1, 3, 4},
        "locations": {2, 3, 6, 7},
    },
    1: {  # car
        "time": {1},
        "weather": {1, 3, 4},
        "locations": {2, 3, 6, 7},
    },
    2: {  # plane
        "time": {1},
        "weather": {1, 3, 4},
        "locations": {0, 2, 3, 4, 6, 7},
    },
    3: {  # ship
        "time": {1},
        "weather": {1, 3, 4},
        "locations": {0, 1, 2, 3, 4, 5, 6},
    },
    4: {  # cat
        "time": {1},
        "weather": {1, 3, 4},
        "locations": {0, 3, 4, 6, 7},
    },
    5: {  # dog
        "time": {1},
        "weather": {1, 3, 4},
        "locations": {0, 3, 6},
    },
    6: {  # horse
        "time": {1},
        "weather": {1, 3, 4},
        "locations": {2, 3, 5, 6, 7},
    },
    7: {  # deer
        "time": {1},
        "weather": {1, 3, 4},
        "locations": {2, 3, 4, 5, 6, 7},
    },
    8: {  # frog
        "time": {},
        "weather": {1, 3, 4},
        "locations": {2, 5, 6},
    },
    9: {  # bird
        "time": {1},
        "weather": {1, 3, 4},
        "locations": {2, 5, 6},
    },
}

uncommon_time_mask = np.zeros((10, 2))
uncommon_weather_mask = np.zeros((10, 6))
uncommon_locations_mask = np.zeros((10, 8))

for i in range(10):
    uncommon_time_mask[i, list(uncommon[i]['time'])] = 1
    uncommon_weather_mask[i, list(uncommon[i]['weather'])] = 1
    uncommon_locations_mask[i, list(uncommon[i]['locations'])] = 1


In [31]:
results = []
for b_img, b_class, b_time, b_weather, b_locations in tqdm(dataloader):
    with torch.no_grad():
        b_pred_class_probs_locations = torch.softmax(model(b_img.to(device), which=0, context=b_locations.argmax(-1)), dim=-1)
        b_pred_class_probs_weather = torch.softmax(model(b_img.to(device), which=1, context=b_weather), dim=-1)
        b_pred_class_probs_time = torch.softmax(model(b_img.to(device), which=2, context=b_time), dim=-1)
    results.append(torch.cat((b_pred_class_probs_locations.cpu(), b_pred_class_probs_weather.cpu(), b_pred_class_probs_time.cpu(), b_class.unsqueeze(-1), b_time.unsqueeze(-1), b_weather.unsqueeze(-1), b_locations) , dim=-1))

  0%|          | 0/94 [00:00<?, ?it/s]

torch.Size([256, 1, 10])
torch.Size([256, 1, 10])


  1%|          | 1/94 [00:17<27:43, 17.89s/it]

torch.Size([256, 1, 10])
torch.Size([256, 1, 10])
torch.Size([256, 1, 10])


  2%|▏         | 2/94 [00:36<28:13, 18.40s/it]

...

torch.Size([256, 1, 10])
torch.Size([256, 1, 10])
torch.Size([256, 1, 10])


 99%|█████████▉| 93/94 [34:04<00:21, 21.70s/it]

torch.Size([256, 1, 10])


100%|██████████| 94/94 [34:13<00:00, 21.84s/it]

torch.Size([94, 1, 10])
torch.Size([94, 1, 10])
torch.Size([94, 1, 10])


In [30]:
b_locations.argmax(-1)

tensor([3, 2, 0, 2, 1, 8, 1, 7, 4, 6, 2, 8, 8, 1, 4, 8, 1, 2, 8, 1, 5, 2, 7, 2,
        2, 7, 1, 4, 7, 5, 0, 1, 8, 0, 5, 2, 1, 5, 5, 4, 7, 2, 8, 2, 3, 8, 2, 0,
        4, 0, 3, 8, 8, 1, 1, 7, 5, 8, 4, 6, 1, 8, 4, 8, 2, 1, 2, 3, 2, 4, 4, 7,
        8, 1, 6, 7, 6, 0, 2, 5, 5, 4, 3, 0, 6, 8, 4, 7, 4, 1, 0, 3, 2, 5, 5, 1,
        8, 0, 4, 5, 4, 5, 5, 2, 0, 8, 1, 1, 2, 1, 0, 8, 2, 7, 0, 6, 5, 1, 7, 0,
        6, 2, 8, 2, 4, 0, 6, 0, 1, 6, 2, 5, 0, 7, 1, 1, 6, 7, 0, 2, 8, 7, 7, 0,
        1, 2, 1, 4, 4, 7, 3, 4, 0, 5, 4, 1, 8, 4, 8, 5, 0, 0, 8, 8, 2, 3, 1, 2,
        7, 6, 7, 4, 2, 2, 5, 1, 1, 8, 0, 0, 7, 3, 8, 1, 8, 7, 0, 4, 0, 8, 1, 7,
        0, 1, 2, 8, 3, 1, 7, 1, 0, 7, 2, 1, 8, 2, 0, 7, 4, 5, 3, 0, 2, 4, 7, 8,
        5, 3, 7, 7, 4, 7, 7, 7, 2, 8, 4, 7, 1, 7, 1, 2, 2, 0, 8, 1, 2, 5, 1, 8,
        2, 8, 7, 5, 8, 0, 1, 0, 1, 1, 8, 1, 7, 0, 7, 6])

In [34]:
results = torch.cat(results , 0)

In [39]:
preds_loc.shape

torch.Size([23902, 10])

In [38]:
# with open('./results_with_context.pkl', 'wb+') as fp:
#     pickle.dump(results, fp)
preds_loc, preds_weather, preds_time, labels, time, weather, location = results[:,:10], results[:,10:20], results[:,20:30], results[:,30], results[:,31], results[:,32], results[:,33:] 

In [45]:
hits_loc = (preds_loc.argmax(-1) == labels).float()

In [43]:
print((preds_weather.argmax(-1) == labels).float().mean())

tensor(0.9341)


In [44]:
print((preds_time.argmax(-1) == labels).float().mean())

tensor(0.9339)


In [57]:
with open('./results.pkl', 'rb') as fp:
     preds = pickle.load(fp)[:,:10]
hits = (preds.argmax(-1) == labels).float()

In [33]:
preds, labels, time, weather, location = results[:,:10], results[:,10], results[:,11], results[:,12], results[:,13:] 

TypeError: list indices must be integers or slices, not tuple

In [21]:
hits = (preds.argmax(-1) == labels).int()

In [22]:
torch.mean(hits.float())

tensor(0.9384)

In [47]:
bg = BGVarDataset

In [62]:
r_mat = np.matrix(np.zeros((len(bg.locations), len(bg.categories))))
l_mat = np.matrix(np.zeros((len(bg.locations), len(bg.categories))))
for lk, lv in bg.locations.items():
    for ck, cv in  bg.categories.items():
        selected_hits = hits_loc[torch.logical_and(location[:,lv] == 1, cv == labels)]
        r_mat[lv, cv] = torch.sum(selected_hits)/len(selected_hits)
        l_mat[lv, cv] = len(selected_hits)

In [61]:
r_mat.std(-1)

matrix([[0.07157264],
        [0.08343141],
        [0.10336953],
        [0.11404232],
        [0.10582133],
        [0.05348636],
        [0.06454713],
        [0.0978155 ],
        [0.07100307]])

In [63]:
r_mat.std(-1)

matrix([[0.06684541],
        [0.07764915],
        [0.11705085],
        [0.11521453],
        [0.1023005 ],
        [0.06743765],
        [0.07492102],
        [0.10112057],
        [0.06725361]])

In [59]:
pd.DataFrame(np.round(r_mat, 3), columns= bg.categories.keys(), index=bg.locations.keys())

,truck,car,plane,ship,cat,dog,horse,deer,frog,bird
forest,0.914,0.742,0.944,0.987,0.889,0.927,0.977,0.983,0.991,0.972
grass,0.946,0.694,0.967,0.969,0.951,0.960,0.984,0.984,0.986,0.972
indoors,0.784,0.865,0.975,1.000,0.981,0.981,0.987,0.667,0.946,0.902
rocks,0.895,0.597,0.923,1.000,0.977,0.974,0.976,0.930,0.996,0.979
sand,0.905,0.615,0.927,0.978,0.911,0.980,0.970,0.935,0.994,0.978
street,0.929,0.790,0.976,0.979,0.920,0.938,0.928,0.986,0.938,0.968
snow,0.885,0.773,0.946,1.000,0.936,0.947,0.982,0.961,1.000,0.974
water,0.785,0.676,0.876,0.993,0.922,0.968,0.961,0.904,0.995,0.972
none,0.811,0.798,0.988,0.969,0.975,0.971,0.984,0.925,0.998,0.993


In [ ]:
(np.mean(np.array(r_mat[:-1,:].transpose()).flatten()[uncommon_locations_mask.flatten().astype(bool)]), np.mean(np.array(r_mat[:-1,:].transpose()).flatten()[~uncommon_locations_mask.flatten().astype(bool)]) )

In [ ]:
r_mat = np.matrix(np.zeros((len(bg.times), len(bg.categories))))
l_mat = np.matrix(np.zeros((len(bg.times), len(bg.categories))))
for lk, lv in bg.times.items():
    for ck, cv in  bg.categories.items():
        selected_hits = hits[torch.logical_and(time == lv, cv == labels)]
        r_mat[lv, cv] = torch.sum(selected_hits)/len(selected_hits)
        l_mat[lv, cv] = len(selected_hits)

In [ ]:
pd.DataFrame(np.round(r_mat, 3), columns=bg.categories.keys(), index=bg.times.keys())

In [ ]:
(np.mean(np.array(r_mat[:-1,:].transpose()).flatten()[uncommon_time_mask.flatten().astype(bool)]), np.mean(np.array(r_mat[:-1,:].transpose()).flatten()[~uncommon_time_mask.flatten().astype(bool)]))

In [ ]:
r_mat = np.matrix(np.zeros((len(bg.weathers), len(bg.categories))))
l_mat = np.matrix(np.zeros((len(bg.weathers), len(bg.categories))))
for lk, lv in bg.weathers.items():
    for ck, cv in  bg.categories.items():
        selected_hits = hits[torch.logical_and(weather == lv, cv == labels)]
        r_mat[lv, cv] = torch.sum(selected_hits)/(len(selected_hits)+1e-6)
        l_mat[lv, cv] = len(selected_hits)

In [ ]:
pd.DataFrame(np.round(r_mat, 3), columns=bg.categories.keys(), index=bg.weathers.keys())

In [ ]:
(np.mean(np.array(r_mat[:-1,:].transpose()).flatten()[uncommon_weather_mask.flatten().astype(bool)]), np.mean(np.array(r_mat[:-1,:].transpose()).flatten()[~uncommon_weather_mask.flatten().astype(bool)]))

In [ ]:
 np.corrcoef(r_mat.flatten(), l_mat.flatten())

In [ ]:
l_mat

In [ ]:
print(mutual_info_score(labels, weather), mutual_info_score(labels, random.sample(list(weather), len(weather))))

In [ ]:
print(mutual_info_score(labels, time), mutual_info_score(labels, random.sample(list(time), len(time))))

In [ ]:
with open('./class_adv_success_rate_textprobe_per_class.pkl', 'rb') as fp:
    class_accuracies = pickle.load(fp)
with open('./all_text_probes_per_class.pkl', 'rb') as fp:
    all_text_probes = pickle.load(fp)
    
raw_images = torchvision.datasets.ImageNet("/fs/classhomes/fall2021/cmsc828w/cs828w04/ILSVRC2012/", split='val', transform=None)

In [ ]:
imagenet_classes = ["tench", "goldfish", "great white shark", "tiger shark", "hammerhead shark", "electric ray", "stingray", "rooster", "hen", "ostrich", "brambling", "goldfinch", "house finch", "junco", "indigo bunting", "American robin", "bulbul", "jay", "magpie", "chickadee", "American dipper", "kite (bird of prey)", "bald eagle", "vulture", "great grey owl", "fire salamander", "smooth newt", "newt", "spotted salamander", "axolotl", "American bullfrog", "tree frog", "tailed frog", "loggerhead sea turtle", "leatherback sea turtle", "mud turtle", "terrapin", "box turtle", "banded gecko", "green iguana", "Carolina anole", "desert grassland whiptail lizard", "agama", "frilled-necked lizard", "alligator lizard", "Gila monster", "European green lizard", "chameleon", "Komodo dragon", "Nile crocodile", "American alligator", "triceratops", "worm snake", "ring-necked snake", "eastern hog-nosed snake", "smooth green snake", "kingsnake", "garter snake", "water snake", "vine snake", "night snake", "boa constrictor", "African rock python", "Indian cobra", "green mamba", "sea snake", "Saharan horned viper", "eastern diamondback rattlesnake", "sidewinder rattlesnake", "trilobite", "harvestman", "scorpion", "yellow garden spider", "barn spider", "European garden spider", "southern black widow", "tarantula", "wolf spider", "tick", "centipede", "black grouse", "ptarmigan", "ruffed grouse", "prairie grouse", "peafowl", "quail", "partridge", "african grey parrot", "macaw", "sulphur-crested cockatoo", "lorikeet", "coucal", "bee eater", "hornbill", "hummingbird", "jacamar", "toucan", "duck", "red-breasted merganser", "goose", "black swan", "tusker", "echidna", "platypus", "wallaby", "koala", "wombat", "jellyfish", "sea anemone", "brain coral", "flatworm", "nematode", "conch", "snail", "slug", "sea slug", "chiton", "chambered nautilus", "Dungeness crab", "rock crab", "fiddler crab", "red king crab", "American lobster", "spiny lobster", "crayfish", "hermit crab", "isopod", "white stork", "black stork", "spoonbill", "flamingo", "little blue heron", "great egret", "bittern bird", "crane bird", "limpkin", "common gallinule", "American coot", "bustard", "ruddy turnstone", "dunlin", "common redshank", "dowitcher", "oystercatcher", "pelican", "king penguin", "albatross", "grey whale", "killer whale", "dugong", "sea lion", "Chihuahua", "Japanese Chin", "Maltese", "Pekingese", "Shih Tzu", "King Charles Spaniel", "Papillon", "toy terrier", "Rhodesian Ridgeback", "Afghan Hound", "Basset Hound", "Beagle", "Bloodhound", "Bluetick Coonhound", "Black and Tan Coonhound", "Treeing Walker Coonhound", "English foxhound", "Redbone Coonhound", "borzoi", "Irish Wolfhound", "Italian Greyhound", "Whippet", "Ibizan Hound", "Norwegian Elkhound", "Otterhound", "Saluki", "Scottish Deerhound", "Weimaraner", "Staffordshire Bull Terrier", "American Staffordshire Terrier", "Bedlington Terrier", "Border Terrier", "Kerry Blue Terrier", "Irish Terrier", "Norfolk Terrier", "Norwich Terrier", "Yorkshire Terrier", "Wire Fox Terrier", "Lakeland Terrier", "Sealyham Terrier", "Airedale Terrier", "Cairn Terrier", "Australian Terrier", "Dandie Dinmont Terrier", "Boston Terrier", "Miniature Schnauzer", "Giant Schnauzer", "Standard Schnauzer", "Scottish Terrier", "Tibetan Terrier", "Australian Silky Terrier", "Soft-coated Wheaten Terrier", "West Highland White Terrier", "Lhasa Apso", "Flat-Coated Retriever", "Curly-coated Retriever", "Golden Retriever", "Labrador Retriever", "Chesapeake Bay Retriever", "German Shorthaired Pointer", "Vizsla", "English Setter", "Irish Setter", "Gordon Setter", "Brittany dog", "Clumber Spaniel", "English Springer Spaniel", "Welsh Springer Spaniel", "Cocker Spaniel", "Sussex Spaniel", "Irish Water Spaniel", "Kuvasz", "Schipperke", "Groenendael dog", "Malinois", "Briard", "Australian Kelpie", "Komondor", "Old English Sheepdog", "Shetland Sheepdog", "collie", "Border Collie", "Bouvier des Flandres dog", "Rottweiler", "German Shepherd Dog", "Dobermann", "Miniature Pinscher", "Greater Swiss Mountain Dog", "Bernese Mountain Dog", "Appenzeller Sennenhund", "Entlebucher Sennenhund", "Boxer", "Bullmastiff", "Tibetan Mastiff", "French Bulldog", "Great Dane", "St. Bernard", "husky", "Alaskan Malamute", "Siberian Husky", "Dalmatian", "Affenpinscher", "Basenji", "pug", "Leonberger", "Newfoundland dog", "Great Pyrenees dog", "Samoyed", "Pomeranian", "Chow Chow", "Keeshond", "brussels griffon", "Pembroke Welsh Corgi", "Cardigan Welsh Corgi", "Toy Poodle", "Miniature Poodle", "Standard Poodle", "Mexican hairless dog (xoloitzcuintli)", "grey wolf", "Alaskan tundra wolf", "red wolf or maned wolf", "coyote", "dingo", "dhole", "African wild dog", "hyena", "red fox", "kit fox", "Arctic fox", "grey fox", "tabby cat", "tiger cat", "Persian cat", "Siamese cat", "Egyptian Mau", "cougar", "lynx", "leopard", "snow leopard", "jaguar", "lion", "tiger", "cheetah", "brown bear", "American black bear", "polar bear", "sloth bear", "mongoose", "meerkat", "tiger beetle", "ladybug", "ground beetle", "longhorn beetle", "leaf beetle", "dung beetle", "rhinoceros beetle", "weevil", "fly", "bee", "ant", "grasshopper", "cricket insect", "stick insect", "cockroach", "praying mantis", "cicada", "leafhopper", "lacewing", "dragonfly", "damselfly", "red admiral butterfly", "ringlet butterfly", "monarch butterfly", "small white butterfly", "sulphur butterfly", "gossamer-winged butterfly", "starfish", "sea urchin", "sea cucumber", "cottontail rabbit", "hare", "Angora rabbit", "hamster", "porcupine", "fox squirrel", "marmot", "beaver", "guinea pig", "common sorrel horse", "zebra", "pig", "wild boar", "warthog", "hippopotamus", "ox", "water buffalo", "bison", "ram (adult male sheep)", "bighorn sheep", "Alpine ibex", "hartebeest", "impala (antelope)", "gazelle", "arabian camel", "llama", "weasel", "mink", "European polecat", "black-footed ferret", "otter", "skunk", "badger", "armadillo", "three-toed sloth", "orangutan", "gorilla", "chimpanzee", "gibbon", "siamang", "guenon", "patas monkey", "baboon", "macaque", "langur", "black-and-white colobus", "proboscis monkey", "marmoset", "white-headed capuchin", "howler monkey", "titi monkey", "Geoffroy's spider monkey", "common squirrel monkey", "ring-tailed lemur", "indri", "Asian elephant", "African bush elephant", "red panda", "giant panda", "snoek fish", "eel", "silver salmon", "rock beauty fish", "clownfish", "sturgeon", "gar fish", "lionfish", "pufferfish", "abacus", "abaya", "academic gown", "accordion", "acoustic guitar", "aircraft carrier", "airliner", "airship", "altar", "ambulance", "amphibious vehicle", "analog clock", "apiary", "apron", "trash can", "assault rifle", "backpack", "bakery", "balance beam", "balloon", "ballpoint pen", "Band-Aid", "banjo", "baluster / handrail", "barbell", "barber chair", "barbershop", "barn", "barometer", "barrel", "wheelbarrow", "baseball", "basketball", "bassinet", "bassoon", "swimming cap", "bath towel", "bathtub", "station wagon", "lighthouse", "beaker", "military hat (bearskin or shako)", "beer bottle", "beer glass", "bell tower", "baby bib", "tandem bicycle", "bikini", "ring binder", "binoculars", "birdhouse", "boathouse", "bobsleigh", "bolo tie", "poke bonnet", "bookcase", "bookstore", "bottle cap", "hunting bow", "bow tie", "brass memorial plaque", "bra", "breakwater", "breastplate", "broom", "bucket", "buckle", "bulletproof vest", "high-speed train", "butcher shop", "taxicab", "cauldron", "candle", "cannon", "canoe", "can opener", "cardigan", "car mirror", "carousel", "tool kit", "cardboard box / carton", "car wheel", "automated teller machine", "cassette", "cassette player", "castle", "catamaran", "CD player", "cello", "mobile phone", "chain", "chain-link fence", "chain mail", "chainsaw", "storage chest", "chiffonier", "bell or wind chime", "china cabinet", "Christmas stocking", "church", "movie theater", "cleaver", "cliff dwelling", "cloak", "clogs", "cocktail shaker", "coffee mug", "coffeemaker", "spiral or coil", "combination lock", "computer keyboard", "candy store", "container ship", "convertible", "corkscrew", "cornet", "cowboy boot", "cowboy hat", "cradle", "construction crane", "crash helmet", "crate", "infant bed", "Crock Pot", "croquet ball", "crutch", "cuirass", "dam", "desk", "desktop computer", "rotary dial telephone", "diaper", "digital clock", "digital watch", "dining table", "dishcloth", "dishwasher", "disc brake", "dock", "dog sled", "dome", "doormat", "drilling rig", "drum", "drumstick", "dumbbell", "Dutch oven", "electric fan", "electric guitar", "electric locomotive", "entertainment center", "envelope", "espresso machine", "face powder", "feather boa", "filing cabinet", "fireboat", "fire truck", "fire screen", "flagpole", "flute", "folding chair", "football helmet", "forklift", "fountain", "fountain pen", "four-poster bed", "freight car", "French horn", "frying pan", "fur coat", "garbage truck", "gas mask or respirator", "gas pump", "goblet", "go-kart", "golf ball", "golf cart", "gondola", "gong", "gown", "grand piano", "greenhouse", "radiator grille", "grocery store", "guillotine", "hair clip", "hair spray", "half-track", "hammer", "hamper", "hair dryer", "hand-held computer", "handkerchief", "hard disk drive", "harmonica", "harp", "combine harvester", "hatchet", "holster", "home theater", "honeycomb", "hook", "hoop skirt", "gymnastic horizontal bar", "horse-drawn vehicle", "hourglass", "iPod", "clothes iron", "carved pumpkin", "jeans", "jeep", "T-shirt", "jigsaw puzzle", "rickshaw", "joystick", "kimono", "knee pad", "knot", "lab coat", "ladle", "lampshade", "laptop computer", "lawn mower", "lens cap", "letter opener", "library", "lifeboat", "lighter", "limousine", "ocean liner", "lipstick", "slip-on shoe", "lotion", "music speaker", "loupe magnifying glass", "sawmill", "magnetic compass", "messenger bag", "mailbox", "tights", "one-piece bathing suit", "manhole cover", "maraca", "marimba", "mask", "matchstick", "maypole", "maze", "measuring cup", "medicine cabinet", "megalith", "microphone", "microwave oven", "military uniform", "milk can", "minibus", "miniskirt", "minivan", "missile", "mitten", "mixing bowl", "mobile home", "ford model t", "modem", "monastery", "monitor", "moped", "mortar and pestle", "graduation cap", "mosque", "mosquito net", "vespa", "mountain bike", "tent", "computer mouse", "mousetrap", "moving van", "muzzle", "metal nail", "neck brace", "necklace", "baby pacifier", "notebook computer", "obelisk", "oboe", "ocarina", "odometer", "oil filter", "pipe organ", "oscilloscope", "overskirt", "bullock cart", "oxygen mask", "product packet / packaging", "paddle", "paddle wheel", "padlock", "paintbrush", "pajamas", "palace", "pan flute", "paper towel", "parachute", "parallel bars", "park bench", "parking meter", "railroad car", "patio", "payphone", "pedestal", "pencil case", "pencil sharpener", "perfume", "Petri dish", "photocopier", "plectrum", "Pickelhaube", "picket fence", "pickup truck", "pier", "piggy bank", "pill bottle", "pillow", "ping-pong ball", "pinwheel", "pirate ship", "drink pitcher", "block plane", "planetarium", "plastic bag", "plate rack", "farm plow", "plunger", "Polaroid camera", "pole", "police van", "poncho", "pool table", "soda bottle", "plant pot", "potter's wheel", "power drill", "prayer rug", "printer", "prison", "missile", "projector", "hockey puck", "punching bag", "purse", "quill", "quilt", "race car", "racket", "radiator", "radio", "radio telescope", "rain barrel", "recreational vehicle", "fishing casting reel", "reflex camera", "refrigerator", "remote control", "restaurant", "revolver", "rifle", "rocking chair", "rotisserie", "eraser", "rugby ball", "ruler measuring stick", "sneaker", "safe", "safety pin", "salt shaker", "sandal", "sarong", "saxophone", "scabbard", "weighing scale", "school bus", "schooner", "scoreboard", "CRT monitor", "screw", "screwdriver", "seat belt", "sewing machine", "shield", "shoe store", "shoji screen / room divider", "shopping basket", "shopping cart", "shovel", "shower cap", "shower curtain", "ski", "balaclava ski mask", "sleeping bag", "slide rule", "sliding door", "slot machine", "snorkel", "snowmobile", "snowplow", "soap dispenser", "soccer ball", "sock", "solar thermal collector", "sombrero", "soup bowl", "keyboard space bar", "space heater", "space shuttle", "spatula", "motorboat", "spider web", "spindle", "sports car", "spotlight", "stage", "steam locomotive", "through arch bridge", "steel drum", "stethoscope", "scarf", "stone wall", "stopwatch", "stove", "strainer", "tram", "stretcher", "couch", "stupa", "submarine", "suit", "sundial", "sunglasses", "sunglasses", "sunscreen", "suspension bridge", "mop", "sweatshirt", "swim trunks / shorts", "swing", "electrical switch", "syringe", "table lamp", "tank", "tape player", "teapot", "teddy bear", "television", "tennis ball", "thatched roof", "front curtain", "thimble", "threshing machine", "throne", "tile roof", "toaster", "tobacco shop", "toilet seat", "torch", "totem pole", "tow truck", "toy store", "tractor", "semi-trailer truck", "tray", "trench coat", "tricycle", "trimaran", "tripod", "triumphal arch", "trolleybus", "trombone", "hot tub", "turnstile", "typewriter keyboard", "umbrella", "unicycle", "upright piano", "vacuum cleaner", "vase", "vaulted or arched ceiling", "velvet fabric", "vending machine", "vestment", "viaduct", "violin", "volleyball", "waffle iron", "wall clock", "wallet", "wardrobe", "military aircraft", "sink", "washing machine", "water bottle", "water jug", "water tower", "whiskey jug", "whistle", "hair wig", "window screen", "window shade", "Windsor tie", "wine bottle", "airplane wing", "wok", "wooden spoon", "wool", "split-rail fence", "shipwreck", "sailboat", "yurt", "website", "comic book", "crossword", "traffic or street sign", "traffic light", "dust jacket", "menu", "plate", "guacamole", "consomme", "hot pot", "trifle", "ice cream", "popsicle", "baguette", "bagel", "pretzel", "cheeseburger", "hot dog", "mashed potatoes", "cabbage", "broccoli", "cauliflower", "zucchini", "spaghetti squash", "acorn squash", "butternut squash", "cucumber", "artichoke", "bell pepper", "cardoon", "mushroom", "Granny Smith apple", "strawberry", "orange", "lemon", "fig", "pineapple", "banana", "jackfruit", "cherimoya (custard apple)", "pomegranate", "hay", "carbonara", "chocolate syrup", "dough", "meatloaf", "pizza", "pot pie", "burrito", "red wine", "espresso", "tea cup", "eggnog", "mountain", "bubble", "cliff", "coral reef", "geyser", "lakeshore", "promontory", "sandbar", "beach", "valley", "volcano", "baseball player", "bridegroom", "scuba diver", "rapeseed", "daisy", "yellow lady's slipper", "corn", "acorn", "rose hip", "horse chestnut seed", "coral fungus", "agaric", "gyromitra", "stinkhorn mushroom", "earth star fungus", "hen of the woods mushroom", "bolete", "corn cob", "toilet paper"]

In [ ]:
ind = 205

In [ ]:
imagenet_classes[ind//50]

In [ ]:
class_accuracies[ind//50]

In [ ]:
# all_text_probes = dict([(x, (y, z)) for x , y, z in all_text_probes])
all_text_probes[ind]

In [ ]:
plt.imshow(raw_images.__getitem__(ind)[0], interpolation='nearest')
plt.show()

In [ ]:
plt.hist(class_accuracies)
plt.xlabel('Success rate in changing label per class')
plt.ylabel('Frequency')
plt.show()

Car class has very poor accuracy